Using the code from the lesson as a guide, create a data frame named items that has all of the data for items.

Do the same thing, but for stores.

Extract the data for sales.

There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

Save the data in your files to local csv files so that it will be faster to access in the future.

Combine the data from your three separate data frames into one large data frame.

Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions and be able to re-run the functions and get the same results.

## Items

In [94]:
import requests
import pandas as pd
import numpy as np

In [2]:
base_url = 'https://python.zach.lol'

In [3]:
response = requests.get(base_url + '/api/v1/items')

In [4]:
data = response.json()

In [5]:
data['payload']['page']

1

In [6]:
# data['payload']['items']

In [7]:
items = data['payload']['items']

In [8]:
items_df = pd.DataFrame(items)

In [9]:
items_df.shape

(20, 6)

In [10]:
data['payload']['next_page']

'/api/v1/items?page=2'

In [11]:
response = requests.get(base_url + data['payload']['next_page'])

In [12]:
data = response.json()

In [13]:
# data['payload']['items']

In [14]:
items += data['payload']['items']

In [15]:
response = requests.get(base_url + data['payload']['next_page'])

In [16]:
data = response.json()

In [17]:
items += data['payload']['items']

In [18]:
items_df = pd.DataFrame(items)

In [19]:
items_df.shape

(50, 6)

In [20]:
items_df.to_csv('items.csv')

## Stores

In [21]:
base_url = 'https://python.zach.lol'

In [22]:
response = requests.get(base_url + '/api/v1/stores')

In [23]:
data = response.json()

In [24]:
stores = data['payload']['stores']

In [25]:
stores_df = pd.DataFrame(stores)

In [26]:
stores_df.shape

(10, 5)

In [27]:
stores_df.to_csv('stores.csv')

## Sales

In [28]:
base_url = 'https://python.zach.lol'

In [29]:
response = requests.get(base_url + '/api/v1/sales')

In [30]:
data = response.json()

In [31]:
sales = data['payload']['sales']

In [42]:
sales_df = pd.DataFrame(sales)

In [43]:
sales_df.shape

(5000, 5)

In [45]:
data['payload']['max_page']

183

In [58]:
def get_sales():
    base_url = 'https://python.zach.lol'
    response = requests.get(base_url + '/api/v1/sales')
    data = response.json()
    sales = data['payload']['sales']
    while data['payload']['page'] < 184:
        response = requests.get(base_url + data['payload']['next_page'])
        data = response.json()
        sales += data['payload']['sales']
        if data['payload']['next_page'] == None:
            break
    return pd.DataFrame(sales)

In [60]:
sales_df = get_sales()

In [61]:
sales_df.shape

(913000, 5)

In [62]:
sales_df.to_csv('sales.csv')

## Combine

In [80]:
sales_df.rename(columns={"store": "store_id"}, inplace=True)

In [104]:
combined_df = sales_df.join(stores_df.set_index('store_id'), on='store_id')
combined_df.head()

,item,sale_amount,sale_date,sale_id,store_id,store_address,store_city,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253


In [88]:
combined_df.rename(columns={"item": "item_id"}, inplace=True)

In [89]:
combined_df = combined_df.join(items_df.set_index('item_id'), on='item_id')

In [105]:
combined_df.head()

,item,sale_amount,sale_date,sale_id,store_id,store_address,store_city,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253


In [103]:
combined_df.to_csv('combined.csv')